In [ ]:
# ============================================================
#  run_simu_ppci_mean_matern52.py  (UPDATED; driver only)
#
#  - X in [0,1]^3 (raw), standardized only for kernel/localization
#  - m_true is infinitely smooth
#  - NEW DGP for auxiliary predictor A(x):
#        Y = m_true(X) + eps
#        A = Y + bias(X) + noise_A
#    so Y - A = -bias(X) - noise_A  (eps cancels)
#  - Budgets mimic census_income code:
#       unlabeled total N_unlab_total = N_t_full (aux) + N_use (inference)
#  - ell tuning: pilot-based ell0_hat -> local LOGELL grid around it
#  - lambda grid: O(1/n_label)
#  - Compare ONLY: PPCI, PPCI_label_only (LO), PPI
#  - Save ONE CSV across all x0
# ============================================================

import os
import sys
import csv
import numpy as np
import cupy as cp

from ppi_py import ppi_mean_ci, ppi_mean_pointestimate

# ---- import YOUR current function package (Matérn 5/2, clean sigma^2, etc.) ----
from conditional_mean_functions import (
    preprocess_X,
    select_logell_by_pilot_logo_cv,
    WeightAtX0,
    ppci_conditional_mean,
    ppci_conditional_mean_label_only,
)

xp = cp
np.set_printoptions(precision=4, suppress=True)


# ============================================================
# 1) Smooth ground-truth conditional mean on [0,1]^3
# ============================================================
def m_true_np(X_raw: np.ndarray) -> np.ndarray:
    """
    Infinitely smooth m(x) on [0,1]^3.
    X_raw: (n,3) numpy
    """
    x1, x2, x3 = X_raw[:, 0], X_raw[:, 1], X_raw[:, 2]
    return (
        np.sin(2 * np.pi * x1)
        + 0.6 * np.cos(2 * np.pi * x2)
        + 0.4 * np.sin(2 * np.pi * x3)
        + 0.25 * np.sin(2 * np.pi * (x1 + x2))
    )


def bias_fn_np(X_raw: np.ndarray, bias_strength: float) -> np.ndarray:
    """
    Smooth systematic bias added to A.
    """
    x1, x2, x3 = X_raw[:, 0], X_raw[:, 1], X_raw[:, 2]
    b = (x1 - 0.5) + 0.5 * np.sin(4 * np.pi * x2) - 0.3 * (x3 - 0.5)
    return bias_strength * b


def A_predict_from_Y_np(
    X_raw: np.ndarray,
    Y: np.ndarray,
    rng: np.random.Generator,
    sigma_A: float,
    bias_strength: float,
) -> np.ndarray:
    """
    NEW auxiliary predictor:
        A = Y + bias(X) + noise_A
    """
    Y = np.asarray(Y, dtype=float).ravel()
    n = int(Y.size)
    noise = rng.normal(0.0, sigma_A, size=n)
    return Y + bias_fn_np(X_raw, bias_strength) + noise


# ============================================================
# 2) x0 list
# ============================================================
def make_x0_grid(low=0.8, high=0.95, num=6):
    """
    Create a 3D grid of test points in [low, high]^3.
    """
    vals = np.linspace(low, high, num)
    x0_list = []
    for a in vals:
        for b in vals:
            for c in vals:
                x0_list.append(np.array([a, b, c], dtype=float))
    return x0_list


# ============================================================
# 3) One-x0 simulation (mirrors census_income structure)
# ============================================================
def simulate_one_x0_from_pool(
    X_raw_total,
    Y_total,
    Yhat_total,
    X_scaled_total,
    X_mean,
    X_std,
    idx_pilot,
    idx_aux,
    idx_eval_pool,
    seed_eval,
    B,
    n_label,
    N_unlab_total,
    N_t_full,
    lam_grid,
    LOGELL_grid_dummy,
    alpha,
    z_alpha,
    x0_raw,
    c_min=0.8,
    c_max=1.2,
):
    rng = np.random.default_rng(seed_eval)

    # budgets
    if N_unlab_total < N_t_full:
        raise ValueError("N_unlab_total must be >= N_t_full.")
    N_use = int(N_unlab_total - N_t_full)

    # x0 scaling (kernel/localization uses scaled X)
    x0_raw = np.asarray(x0_raw, dtype=float).ravel()
    x0_scaled = (x0_raw - X_mean) / X_std

    # true target at x0 (raw scale)
    theta0 = float(m_true_np(x0_raw.reshape(1, 3))[0])

    # -------- pilot (fixed across x0) --------
    X_pilot_scaled = X_scaled_total[idx_pilot]
    Y_pilot = Y_total[idx_pilot]

    # local ell0_hat from pilot distances to x0
    d_pilot = np.linalg.norm(X_pilot_scaled - x0_scaled.reshape(1, 3), axis=1)
    ell0_hat = float(np.median(d_pilot))
    ell0_hat = max(1e-8, ell0_hat)

    ell_low = max(1e-8, c_min * ell0_hat)
    ell_high = c_max * ell0_hat

    # build local LOGELL grid (only grid SIZE uses dummy)
    n_grid = int(len(LOGELL_grid_dummy)) if (LOGELL_grid_dummy is not None) else 50
    LOGELL_grid_local = np.linspace(np.log(ell_low), np.log(ell_high), n_grid)

    # pilot CV (use LOGO with unique groups => effectively LOOCV)
    group_labels = np.arange(len(idx_pilot), dtype=int)

    # move pilot to GPU (your functions use xp=cp)
    X_pilot_gpu = cp.asarray(X_pilot_scaled)
    Y_pilot_gpu = cp.asarray(Y_pilot)

    ell_star, pilot_table = select_logell_by_pilot_logo_cv(
        LOGELL_grid=LOGELL_grid_local,
        X_pilot=X_pilot_gpu,
        Y_pilot=Y_pilot_gpu,
        group_labels=group_labels,
    )

    # -------- aux (Nt) to build f_hat and lambda_hat --------
    X_aux_scaled = X_scaled_total[idx_aux]
    X_aux_gpu = cp.asarray(X_aux_scaled)

    locw = WeightAtX0(X_aux=X_aux_gpu, x0=x0_scaled, ell=ell_star)
    lam_hat, alpha_hat, lam_grid_sorted = locw.select_lambda_lcurve(
        lam_grid=lam_grid, normalize=False, make_plot=False
    )
    f_hat = locw.make_f_hat(alpha_hat)

    # -------- MC storage --------
    thetas_ppci, covered_ppci, widths_ppci = [], [], []
    s2_YmA_list, s2_A_list = [], []

    thetas_lo, covered_lo, widths_lo = [], [], []
    s2_Y_list = []

    thetas_ppi, covered_ppi, widths_ppi = [], [], []

    for _ in range(B):
        # DIRECT DRAW (no eval pool):
        # labeled
        X_l_raw = rng.uniform(0.0, 1.0, size=(n_label, 3))
        eps_l = rng.normal(0.0, sigma_eps, size=n_label)
        Y_l_cpu = m_true_np(X_l_raw) + eps_l
        A_l_cpu = A_predict_from_Y_np(
            X_raw=X_l_raw,
            Y=Y_l_cpu,
            rng=rng,
            sigma_A=sigma_A,
            bias_strength=bias_strength,
        )

        # unlabeled (need Y_u to form A_u, then discard Y_u)
        X_u_raw = rng.uniform(0.0, 1.0, size=(N_use, 3))
        eps_u = rng.normal(0.0, sigma_eps, size=N_use)
        Y_u_cpu = m_true_np(X_u_raw) + eps_u
        A_u_cpu = A_predict_from_Y_np(
            X_raw=X_u_raw,
            Y=Y_u_cpu,
            rng=rng,
            sigma_A=sigma_A,
            bias_strength=bias_strength,
        )

        # standardize using the SAME (X_mean, X_std) from preprocess_X
        X_l_scaled = (X_l_raw - X_mean) / X_std
        X_u_scaled = (X_u_raw - X_mean) / X_std

        # GPU arrays
        X_l = cp.asarray(X_l_scaled)
        Y_l = cp.asarray(Y_l_cpu)
        A_l = cp.asarray(A_l_cpu)

        X_u = cp.asarray(X_u_scaled)
        A_u = cp.asarray(A_u_cpu)

        # --- PPCI (full) ---
        th, se, (lo, up), ex = ppci_conditional_mean(
            X_l=X_l, Y_l=Y_l, X_u=X_u, f_hat=f_hat, A_l=A_l, A_u=A_u,
            z_alpha=z_alpha, return_extras=True
        )
        thetas_ppci.append(float(th))
        covered_ppci.append(float(lo <= theta0 <= up))
        widths_ppci.append(float(up - lo))
        s2_YmA_list.append(ex["sigma2_Y_minus_A"])
        s2_A_list.append(ex["sigma2_A"])

        # --- Label-only (LO) ---
        th2, se2, (lo2, up2), ex2 = ppci_conditional_mean_label_only(
            X_l=X_l, Y_l=Y_l, f_hat=f_hat, z_alpha=z_alpha, return_extras=True
        )
        thetas_lo.append(float(th2))
        covered_lo.append(float(lo2 <= theta0 <= up2))
        widths_lo.append(float(up2 - lo2))
        s2_Y_list.append(ex2["sigma2_Y"])

        # --- PPI (global mean; CPU) ---
        Y_l_cpu2 = cp.asnumpy(Y_l).ravel()
        A_l_cpu2 = cp.asnumpy(A_l).ravel()
        A_u_cpu2 = cp.asnumpy(A_u).ravel()

        ppi_lo, ppi_up = ppi_mean_ci(Y_l_cpu2, A_l_cpu2, A_u_cpu2, alpha=alpha)
        ppi_th = ppi_mean_pointestimate(Y_l_cpu2, A_l_cpu2, A_u_cpu2)

        thetas_ppi.append(float(ppi_th))
        covered_ppi.append(float(ppi_lo <= theta0 <= ppi_up))
        widths_ppi.append(float(ppi_up - ppi_lo))

    # summarize
    t_ppci = np.asarray(thetas_ppci, dtype=float)
    c_ppci = np.asarray(covered_ppci, dtype=float)
    w_ppci = np.asarray(widths_ppci, dtype=float)

    t_lo = np.asarray(thetas_lo, dtype=float)
    c_lo = np.asarray(covered_lo, dtype=float)
    w_lo = np.asarray(widths_lo, dtype=float)

    t_ppi = np.asarray(thetas_ppi, dtype=float)
    c_ppi = np.asarray(covered_ppi, dtype=float)
    w_ppi = np.asarray(widths_ppi, dtype=float)

    s2_YmA = np.asarray(s2_YmA_list, dtype=float)
    s2_A = np.asarray(s2_A_list, dtype=float)
    s2_Y = np.asarray(s2_Y_list, dtype=float)

    out = {
        # x0 / truth
        "x0_1": float(x0_raw[0]),
        "x0_2": float(x0_raw[1]),
        "x0_3": float(x0_raw[2]),
        "theta0": float(theta0),

        # tuning diagnostics
        "c_min": float(c_min),
        "c_max": float(c_max),
        "ell0_hat": float(ell0_hat),
        "ell_low": float(ell_low),
        "ell_high": float(ell_high),
        "ell_star": float(ell_star),
        "lambda_hat": float(lam_hat),

        # budgets
        "B": int(B),
        "n_label": int(n_label),
        "n_pilot": int(len(idx_pilot)),
        "N_unlab_total": int(N_unlab_total),
        "N_t_full": int(N_t_full),
        "N_use": int(N_use),

        # PPCI
        "PPCI_theta_mean": float(t_ppci.mean()),
        "PPCI_theta_rmse": float(np.sqrt(np.mean((t_ppci - theta0) ** 2))),
        "PPCI_coverage": float(c_ppci.mean()),
        "PPCI_avg_ci_width": float(w_ppci.mean()),
        "PPCI_sigma2_Y_minus_A_mean": float(s2_YmA.mean()),
        "PPCI_sigma2_A_mean": float(s2_A.mean()),
        "PPCI_sigma2_Y_minus_A_std": float(s2_YmA.std(ddof=1)) if B > 1 else 0.0,
        "PPCI_sigma2_A_std": float(s2_A.std(ddof=1)) if B > 1 else 0.0,

        # LO
        "LO_theta_mean": float(t_lo.mean()),
        "LO_theta_rmse": float(np.sqrt(np.mean((t_lo - theta0) ** 2))),
        "LO_coverage": float(c_lo.mean()),
        "LO_avg_ci_width": float(w_lo.mean()),
        "LO_sigma2_Y_mean": float(s2_Y.mean()),
        "LO_sigma2_Y_std": float(s2_Y.std(ddof=1)) if B > 1 else 0.0,

        # PPI
        "PPI_theta_mean": float(t_ppi.mean()),
        "PPI_theta_rmse": float(np.sqrt(np.mean((t_ppi - theta0) ** 2))),
        "PPI_coverage": float(c_ppi.mean()),
        "PPI_avg_ci_width": float(w_ppi.mean()),
    }
    return out


def write_results_to_csv(results_list, csv_path):
    if len(results_list) == 0:
        raise ValueError("results_list is empty; nothing to write.")
    fieldnames = list(results_list[0].keys())
    with open(csv_path, "w", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results_list:
            writer.writerow(row)
    print(f"\n[Saved] {csv_path}")


# ============================================================
# 4) Main
# ============================================================
if __name__ == "__main__":
    # choose GPU device here
    cp.cuda.Device(0).use()

    # --------------------------
    # Global config (edit here)
    # --------------------------
    seed_data = 2025
    seed_split = 2026

    B = 500
    n_label = 200
    n_pilot = 200

    N_unlab_total = 7000
    N_t_full = 5000  # aux taken from unlabeled

    sigma_eps = 2.0  # noise std for Y
    sigma_A = 1.2     # extra noise std for A (on top of Y)
    bias_strength = 0.2

    alpha = 0.05
    z_alpha = 1.96

    # lambda grid MUST be O(1/n_label) (your rule)
    lam_grid = np.logspace(np.log10(0.1 / n_label), np.log10(10.0 / n_label), 100)

    # only used for the GRID SIZE
    LOGELL_grid_dummy = np.linspace(np.log(0.2), np.log(1.0), 50)

    # x0 set
    x0_list = make_x0_grid(low=0.70, high=0.85, num=10)  # 1000 points

    # pool sizes (KEEPED, but now ONLY used to build pilot+aux; MC draws are direct)
    N_use = int(N_unlab_total - N_t_full)
    N_eval_pool = 0
    N_total_pool = n_pilot + N_t_full + N_eval_pool

    print("=" * 80)
    print("Simulation pool sizes:")
    print(f"  N_total_pool = {N_total_pool}")
    print(f"  n_pilot      = {n_pilot}")
    print(f"  N_t_full     = {N_t_full}")
    print(f"  N_eval_pool  = {N_eval_pool}")
    print(f"  per-rep draw = n_label({n_label}) + N_use({N_use}) = {n_label + N_use}")
    print("=" * 80)

    # --------------------------
    # Generate ONE global pool (raw scale) for PILOT + AUX ONLY
    #   Y = m(X) + eps
    #   A = Y + bias(X) + noise_A
    # --------------------------
    rng = np.random.default_rng(seed_data)

    X_raw_total = rng.uniform(0.0, 1.0, size=(N_total_pool, 3))
    eps = rng.normal(0.0, sigma_eps, size=N_total_pool)

    Y_total = m_true_np(X_raw_total) + eps
    Yhat_total = A_predict_from_Y_np(
        X_raw=X_raw_total,
        Y=Y_total,
        rng=rng,
        sigma_A=sigma_A,
        bias_strength=bias_strength,
    )

    # standardize X (CPU) for kernel/localization only
    X_scaled_total, X_mean, X_std = preprocess_X(X_raw_total)

    # split indices: pilot / aux / eval_pool (fixed)
    perm_all = rng.permutation(N_total_pool)
    idx_pilot = perm_all[:n_pilot]
    idx_aux = perm_all[n_pilot : n_pilot + N_t_full]
    idx_eval_pool = perm_all[n_pilot + N_t_full :]

    # --------------------------
    # Run all x0
    # --------------------------
    all_rows = []
    for j, x0_raw in enumerate(x0_list, 1):
        print("\n" + "-" * 80)
        print(f"[x0 #{j}/{len(x0_list)}] x0 = {x0_raw}")

        out = simulate_one_x0_from_pool(
            X_raw_total=X_raw_total,
            Y_total=Y_total,
            Yhat_total=Yhat_total,
            X_scaled_total=X_scaled_total,
            X_mean=X_mean,
            X_std=X_std,
            idx_pilot=idx_pilot,
            idx_aux=idx_aux,
            idx_eval_pool=idx_eval_pool,
            seed_eval=seed_split + j,   # different resampling seed per x0
            B=B,
            n_label=n_label,
            N_unlab_total=N_unlab_total,
            N_t_full=N_t_full,
            lam_grid=lam_grid,
            LOGELL_grid_dummy=LOGELL_grid_dummy,
            alpha=alpha,
            z_alpha=z_alpha,
            x0_raw=x0_raw,
            c_min=0.01,
            c_max=0.1,
        )
        all_rows.append(out)

        print(
            f"theta0={out['theta0']:.6f}  "
            f"ell0_hat={out['ell0_hat']:.4f}  ell_star={out['ell_star']:.4f}  "
            f"lambda_hat={out['lambda_hat']:.6g}"
        )
        print(
            "PPCI: "
            f"rmse={out['PPCI_theta_rmse']:.6f}  cov={out['PPCI_coverage']:.3f}  "
            f"width={out['PPCI_avg_ci_width']:.6f}  "
            f"s2_YmA={out['PPCI_sigma2_Y_minus_A_mean']:.4f}  "
            f"s2_A={out['PPCI_sigma2_A_mean']:.4f}"
        )
        print(
            "LO  : "
            f"rmse={out['LO_theta_rmse']:.6f}  cov={out['LO_coverage']:.3f}  "
            f"width={out['LO_avg_ci_width']:.6f}  "
            f"s2_Y={out['LO_sigma2_Y_mean']:.4f}"
        )
        print(
            "PPI : "
            f"rmse={out['PPI_theta_rmse']:.6f}  cov={out['PPI_coverage']:.3f}  "
            f"width={out['PPI_avg_ci_width']:.6f}"
        )

    # save one CSV
    save_dir = os.path.join(os.getcwd(), "results")
    os.makedirs(save_dir, exist_ok=True)
    csv_path = os.path.join(save_dir, "simu_ppci_mean_matern52_all_x0.csv")
    write_results_to_csv(all_rows, csv_path)

    print("\nDone.")
    print(f"CSV saved to: {csv_path}")


/home/ysui8/yangsuienv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Simulation pool sizes:
  N_total_pool = 5200
  n_pilot      = 200
  N_t_full     = 5000
  N_eval_pool  = 0
  per-rep draw = n_label(200) + N_use(2000) = 2200

--------------------------------------------------------------------------------
[x0 #1/1000] x0 = [0.7 0.7 0.7]
theta0=-1.369943  ell0_hat=2.0695  ell_star=0.2069  lambda_hat=0.00424877
PPCI: rmse=0.593057  cov=0.936  width=2.086289  s2_YmA=0.0002  s2_A=0.0006
LO  : rmse=0.818197  cov=0.822  width=2.378363  s2_Y=0.0003
PPI : rmse=1.371447  cov=0.000  width=0.334750

--------------------------------------------------------------------------------
[x0 #2/1000] x0 = [0.7    0.7    0.7167]
theta0=-1.380779  ell0_hat=2.0801  ell_star=0.2080  lambda_hat=0.00424877
PPCI: rmse=0.616141  cov=0.940  width=2.089345  s2_YmA=0.0002  s2_A=0.0006
LO  : rmse=0.779196  cov=0.804  width=2.259113  s2_Y=0.0003
PPI : rmse=1.382847  cov=0.000  width=0.336168

--------------------------------------------------------------------------------
[x0 #3/1000